In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from tqdm import tqdm
import numpy as np
import scipy.sparse as sp
from scipy.sparse import diags
from scipy.linalg import lu_factor, lu_solve
import matplotlib.pyplot as plt
from functools import lru_cache

In [15]:
def beta_matrix(w, d, t, e, m):
    dim = 2 * m

    w = w[:,None,None]

    base_val = (w + 1j*d)

    base = base_val * np.eye(2*m)[None,:,:]


    idy = np.arange(0,2*m-1,1)

    base[:,idy,idy+1] = t
    base[:,idy+1,idy] = t

    idx = np.arange(0,m,2)

    base[:,idx,2*m-1-idx] = t+2
    base[:,2*m-1-idx,idx] = t+2


    return base 

In [16]:
wran = np.arange(0,3,0.01)
beta_matrix(wran, 0.01,1,0,5)

array([[[0.  +0.01j, 1.  +0.j  , 0.  +0.j  , ..., 0.  +0.j  ,
         0.  +0.j  , 3.  +0.j  ],
        [1.  +0.j  , 0.  +0.01j, 1.  +0.j  , ..., 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 1.  +0.j  , 0.  +0.01j, ..., 3.  +0.j  ,
         0.  +0.j  , 0.  +0.j  ],
        ...,
        [0.  +0.j  , 0.  +0.j  , 3.  +0.j  , ..., 0.  +0.01j,
         1.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , ..., 1.  +0.j  ,
         0.  +0.01j, 1.  +0.j  ],
        [3.  +0.j  , 0.  +0.j  , 0.  +0.j  , ..., 0.  +0.j  ,
         1.  +0.j  , 0.  +0.01j]],

       [[0.01+0.01j, 1.  +0.j  , 0.  +0.j  , ..., 0.  +0.j  ,
         0.  +0.j  , 3.  +0.j  ],
        [1.  +0.j  , 0.01+0.01j, 1.  +0.j  , ..., 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 1.  +0.j  , 0.01+0.01j, ..., 3.  +0.j  ,
         0.  +0.j  , 0.  +0.j  ],
        ...,
        [0.  +0.j  , 0.  +0.j  , 3.  +0.j  , ..., 0.01+0.01j,
         1.  +0.j  , 0.  +0.j  ],
        [0. 

In [4]:
non = np.nonzero(beta_matrix(0,0.0,0,0,5))

list(zip(list(non[0]),list(non[1])))

[(np.int64(0), np.int64(9)),
 (np.int64(2), np.int64(7)),
 (np.int64(4), np.int64(5)),
 (np.int64(5), np.int64(4)),
 (np.int64(7), np.int64(2)),
 (np.int64(9), np.int64(0))]

In [ ]:
def T1_matrix(t, m):
    dim = 2 * m
    T = np.zeros((dim, dim), dtype=np.complex128)

    n = np.arange(1, (m - 1)//2 + 1)
    i = 2*n - 1           # to 0-based
    j = 2*m - 2*n

    T[i, j] = t
    return T


In [34]:

def leads_vectorized(w_vals, d, t, e, n, tol=1e-6, max_iter=200000):
    B = len(w_vals)
    unit_batch = beta_matrix( w_vals, d, t, e,n)  # (B, dim, dim) if unitcell uses 2*n
    lu_factors = [lu_factor(unit_batch[i]) for i in range(B)]
    g = np.stack([lu_solve(lu_factors[i], np.eye(unit_batch.shape[-1])) for i in range(B)])

    G = g.copy()
    hopp = T1_matrix(t,n)
    hoppT = hopp.T

    dim = g.shape[-1]
    iden = np.eye(dim, dtype=g.dtype)[None, :, :]  # broadcasts to (B, dim, dim)

    diff = np.inf
    count = 0
    pbar = tqdm(total=0, desc="Dyson iteration", leave=True)
    while diff > tol and count < max_iter:
        A = iden - g @ hoppT @ G @ hopp
        luA = [lu_factor(A[i]) for i in range(B)]
        G_new = np.stack([lu_solve(luA[i], g[i]) for i in range(B)])
        diff = np.max(np.abs(G_new - G))
        G = G_new
        count += 1
        pbar.update(1)
        pbar.set_postfix({"diff": diff})
    pbar.close()
    return G, count


In [35]:
def compute(n):
    w_vals = np.arange(0, 3, 0.01)

    G_all, iterations = leads_vectorized(
        w_vals=w_vals,
        d=1e-4,
        t=1,
        e=0,
        n=n
    )


    return G_all

In [ ]:
compute(7)